<a href="https://colab.research.google.com/github/madhurima5978/Fine-tune-Bert/blob/main/sentimentAnalysisAbsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the necessary libraries

from bs4 import BeautifulSoup as bs #bs4 stands for Beautifulsoup4

import requests

name_list = []
rating_list = []
review_list = []

for page in range(1,1105):

    url = "https://www.flipkart.com/apple-iphone-11-black-64-gb/product-reviews/itm4e5041ba101fd?pid=MOBFWQ6BXGJCEYNY&page="+str(page)
    status_code = requests.get(url)

    #get() sends a HTTP GET request and receive a response

    #get() returns a HTTP status code of response

    #If status code is 200 then OK
    #If status code is 404 then page not found
    #If status code is 503 for server unavailable currently, etc

    if str(status_code) == "<Response [200]>":

        soup = bs(status_code.content,'html.parser') # soup is an object of bs

        print("Started to scrape page ",page)

    #From inspect, click on select option (top right corner in inpect tab)
    #then click on any name of customer on page to get the name tag and it's class name


        names = soup.find_all('p',class_='_2sc7ZR _2V5EHH')

        # same with ratings and reviews

        ratings = soup.find_all('div',class_='_3LWZlK _1BLPMq')
        reviews = soup.find_all('div',class_='t-ZTKy')

        #find_all() will give all the content of specified tag and class

        for i in range(len(names)):
            name_list.append(names[i].get_text())
            if len(ratings) > i:
                rating_list.append(ratings[i].get_text())
            else:
                rating_list.append("N/A")
            if len(reviews) > i:
                review_list.append(reviews[i].get_text())
            else:
                review_list.append("N/A")

        print("Page ",page," scraped succesfully")


In [ ]:
pip install pandas

In [ ]:
# now we'll import pandas to convert the data into a csv
import pandas as pd

df = pd.DataFrame()

df['Customer names'] = name_list
df['Ratings out of 5'] = rating_list
df['Review'] = review_list
df

In [ ]:
df.to_csv(r'iPhone_11_Reviews.csv',index=False)

In [ ]:
# Install dependencies
!pip install torch
!pip install transformers
!pip install datasets
# Fine-tune BERT for aspect-based sentiment analysis


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import torch

# Load the dataset
dataset = load_dataset("jakartaresearch/semeval-absa", 'laptop', split="train")



Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
import torch

# Load the dataset
dataset = load_dataset("jakartaresearch/semeval-absa", 'laptop', split="train")

# Extract texts and aspect information from the dataset
train_texts = dataset['text']
aspect_data = dataset['aspects']

# Extract aspects, polarities, and character indices from aspect_data
train_aspects = [item['term'] for item in aspect_data]
train_polarities = [item['polarity'] for item in aspect_data]
train_from_indices = [int(item['from'][0]) for item in aspect_data]  # Convert to integer
train_to_indices = [int(item['to'][0]) for item in aspect_data]  # Convert to integer
num_classes = 4
label_mappings = {"positive": 0, "negative": 1, "neutral": 2, "conflict": 3}

# Tokenize and preprocess your sentences, aspects, and polarities
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenized_data = []
for text, aspect, polarities, from_idx, to_idx in zip(train_texts, train_aspects, train_polarities, train_from_indices, train_to_indices):
    # Extract the aspect from the text using character indices
    extracted_aspect = text[from_idx:to_idx]

    # Combine the sentence and aspect
    combined_text = f"{text} [SEP] {extracted_aspect}"

    # Tokenize the combined text
    inputs = tokenizer(combined_text, return_tensors='pt', padding='max_length', truncation=True, max_length=256)  # Adjust max_length as needed

    # Map polarities to numerical values (handle empty polarity by assigning a default value)
    polarity = polarities[0] if polarities else 'neutral'  # Assign 'neutral' as default
    label = torch.tensor(label_mappings.get(polarity, label_mappings['neutral']))

    tokenized_data.append((inputs, label))

# Convert the tokenized data to tensors
input_tensors = {'input_ids': torch.stack([item[0]['input_ids'].squeeze(0) for item in tokenized_data]),
                 'attention_mask': torch.stack([item[0]['attention_mask'].squeeze(0) for item in tokenized_data])}
labels = torch.stack([item[1] for item in tokenized_data])

# Fine-tune BERT for aspect-based sentiment analysis
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Convert your dataset to PyTorch DataLoader
train_dataset = TensorDataset(input_tensors['input_ids'], input_tensors['attention_mask'], labels)
batch_size = 8  # Adjust as needed
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Continue with the rest of the steps for training and evaluation
# ...


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have defined your model architecture and dataloaders as mentioned before
# ...

# Fine-tune BERT for aspect-based sentiment analysis
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training
epochs = 5  # Adjust as needed
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} - Average Loss: {average_loss}")

# Evaluation
model.eval()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5 - Average Loss: 0.6408798310935028
Epoch 2/5 - Average Loss: 0.36389166904672043
Epoch 3/5 - Average Loss: 0.23899628048068464
Epoch 4/5 - Average Loss: 0.14805053355076944
Epoch 5/5 - Average Loss: 0.07436928955662087


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Save the tokenizer during training
tokenizer.save_pretrained("absa")


('absa/tokenizer_config.json',
 'absa/special_tokens_map.json',
 'absa/vocab.txt',
 'absa/added_tokens.json')

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer
from datasets import load_dataset

test_dataset = load_dataset("jakartaresearch/semeval-absa", 'laptop', split="validation")

# Extract texts and aspect information from the dataset
test_texts = test_dataset['text']
aspect_data_test = test_dataset['aspects']

# Extract aspects, polarities, and character indices from aspect_data
test_aspects = [item['term'] for item in aspect_data_test]
test_polarities = [item['polarity'] for item in aspect_data_test]
test_from_indices = [int(item['from'][0]) for item in aspect_data_test]  # Convert to integer
test_to_indices = [int(item['to'][0]) for item in aspect_data_test]  # Convert to integer

# Tokenize and preprocess your test sentences, aspects, and polarities
tokenized_data_test = []
for text, aspect, polarities, from_idx, to_idx in zip(test_texts, test_aspects, test_polarities, test_from_indices, test_to_indices):
    # Extract the aspect from the text using character indices
    extracted_aspect = text[from_idx:to_idx]

    # Combine the sentence and aspect
    combined_text = f"{text} [SEP] {extracted_aspect}"

    # Tokenize the combined text
    inputs_test = tokenizer(combined_text, return_tensors='pt', padding='max_length', truncation=True, max_length=256)  # Adjust max_length as needed

    # Map polarities to numerical values (handle empty polarity by assigning a default value)
    polarity_test = polarities[0] if polarities else 'neutral'  # Assign 'neutral' as default
    label_test = torch.tensor(label_mappings.get(polarity_test, label_mappings['neutral']))

    tokenized_data_test.append((inputs_test, label_test))

# Convert the tokenized test data to tensors
input_tensors_test = {'input_ids': torch.stack([item[0]['input_ids'].squeeze(0) for item in tokenized_data_test]),
                      'attention_mask': torch.stack([item[0]['attention_mask'].squeeze(0) for item in tokenized_data_test])}
labels_test = torch.stack([item[1] for item in tokenized_data_test])

# Create DataLoader for testing
test_dataset = TensorDataset(input_tensors_test['input_ids'], input_tensors_test['attention_mask'], labels_test)
batch_size_test = 8  # Adjust as needed
test_dataloader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)

# Evaluation loop
# Evaluation loop
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
classification_report_str = classification_report(all_labels, all_preds, target_names=label_mappings.keys())

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report_str)
# Print the logits for each class
softmax_probs = torch.nn.functional.softmax(logits, dim=1)
print(f"Class Probabilities: {softmax_probs}")



Accuracy: 0.87
Classification Report:
              precision    recall  f1-score   support

    positive       0.86      0.89      0.88       222
    negative       0.63      0.79      0.71        92
     neutral       0.96      0.88      0.92       474
    conflict       0.35      0.50      0.41        12

    accuracy                           0.87       800
   macro avg       0.70      0.77      0.73       800
weighted avg       0.88      0.87      0.87       800

Class Probabilities: tensor([[9.8967e-01, 1.4463e-03, 4.0532e-03, 4.8308e-03],
        [8.5580e-01, 1.8639e-02, 1.3541e-02, 1.1202e-01],
        [9.8917e-01, 1.8624e-03, 3.6854e-03, 5.2776e-03],
        [5.3001e-01, 3.6359e-02, 1.7155e-02, 4.1648e-01],
        [1.8649e-04, 3.4201e-04, 9.9922e-01, 2.5085e-04],
        [8.9357e-01, 3.8930e-03, 7.4684e-02, 2.7853e-02],
        [5.5073e-03, 7.0864e-01, 1.1410e-01, 1.7175e-01],
        [7.4535e-02, 2.2010e-01, 1.1594e-02, 6.9377e-01]], device='cuda:0')


In [ ]:
new_sentence =  "The battery life is good but performance is low"

# Tokenize and preprocess the new sentence
tokenized_input = tokenizer(new_sentence, return_tensors="pt", truncation=True, padding=True)

# Move the input to the appropriate device (CPU or GPU)
input_ids = tokenized_input["input_ids"].to(device)
attention_mask = tokenized_input["attention_mask"].to(device)

# Move the model to the appropriate device
model.to(device)

# Ensure the model is in evaluation mode
model.eval()

# Pass the input through the model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Extract the logits from the model's output
logits = outputs.logits

# Post-process the logits to obtain predictions
predictions = torch.argmax(logits, dim=1).item()

# Print the predicted sentiment
sentiment_mapping = {0: "positive", 1: "negative", 2: "neutral", 3: "conflict"}
predicted_sentiment = sentiment_mapping[predictions]
print(f"Predicted Sentiment: {predicted_sentiment}")


Predicted Sentiment: conflict


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset

# Load the trained ABSA model for multiple aspects
model = BertForSequenceClassification.from_pretrained("absa")
tokenizer = BertTokenizer.from_pretrained("absa")

# List of aspects you want to analyze
aspects_to_analyze = ["performance", "battery", "design", "camera"]  # Add your aspects

# Function to perform ABSA on a given text for multiple aspects
def perform_absa_for_aspects(text):
    results = {}

    for aspect in aspects_to_analyze:
        aspect_text = f"{aspect} {text}"  # Concatenate aspect with the original text
        tokenized_input = tokenizer(aspect_text, return_tensors="pt", truncation=True, padding=True)
        input_ids = tokenized_input["input_ids"]
        attention_mask = tokenized_input["attention_mask"]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).item()

        sentiment_mapping = {0: "positive", 1: "negative", 2: "neutral", 3: "conflict"}
        predicted_sentiment = sentiment_mapping[predictions]

        results[aspect] = predicted_sentiment

    return results

# Load the CSV file
csv_path = r'iPhone_11_Reviews.csv'
df = pd.read_csv(csv_path)

# Perform ABSA for each review and each aspect
aspect_sentiments = df['Review'].apply(perform_absa_for_aspects).apply(pd.Series)

# Concatenate the aspect sentiments with the original DataFrame
df = pd.concat([df, aspect_sentiments], axis=1)

# Save the DataFrame with aspect sentiments to a new CSV file
df.to_csv(r'iPhone_11_Reviews_with_Aspect_Sentiments.csv', index=False)


In [19]:
aspect_sentiments = df['Review'].apply(perform_absa_for_aspects).apply(pd.Series)

# Concatenate the aspect sentiments with the original DataFrame
df = pd.concat([df, aspect_sentiments], axis=1)

# Function to combine aspect sentiments into an overall sentiment
def combine_aspect_sentiments(row):
    aspect_sentiments = row[aspects_to_analyze]
    majority_sentiment = aspect_sentiments.mode().iloc[0]  # Get the mode (majority sentiment)
    return majority_sentiment

# Apply the function to each row to get the overall sentiment
df['Overall_Sentiment'] = df.apply(combine_aspect_sentiments, axis=1)

# Save the DataFrame with aspect sentiments and overall sentiment to a new CSV file
df.to_csv(r'iPhone_11_Reviews_with_Aspect_and_Overall_Sentiments.csv', index=False)
df

,Customer names,Ratings out of 5,Review,performance,battery,design,camera,performance,battery,design,camera,Overall_Sentiment
0,siva sankar,5.0,Photos superREAD MORE,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,Steevan Rattra,5.0,Good CameraREAD MORE,positive,positive,positive,positive,positive,positive,positive,positive,positive
2,Abhishek Sachdeva,5.0,Feeling awesome after getting the delivery of ...,positive,positive,positive,positive,positive,positive,positive,positive,positive
3,Vishnu Ramakrishnan,5.0,Camera is awesomeBest battery backupA performe...,positive,neutral,positive,neutral,positive,neutral,positive,neutral,neutral
4,Manish Deshlahare,5.0,Very very goodREAD MORE,positive,neutral,positive,positive,positive,neutral,positive,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
915,Rajeev Ranjan,NaN,Very nice performance. Battery backup is reall...,positive,positive,positive,positive,positive,positive,positive,positive,positive
916,Preeti Nagar,5.0,lovely phone...worth every penny...happy with ...,positive,neutral,neutral,neutral,positive,neutral,neutral,neutral,neutral
917,Abhinav pratap Singh,5.0,Love itREAD MORE,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
918,Arjun c ramesh Ramesh,3.0,"The phone is worth every penny, but it took 2...",negative,negative,negative,negative,negative,negative,negative,negative,negative


In [20]:
df = pd.concat([df, aspect_sentiments], axis=1)

# Function to combine aspect sentiments into an overall sentiment
def combine_aspect_sentiments(row):
    aspect_sentiments = row[aspects_to_analyze]
    majority_sentiment = aspect_sentiments.mode().iloc[0]  # Get the mode (majority sentiment)
    return majority_sentiment

# Apply the function to each row to get the overall sentiment
df['Overall_Sentiment'] = df.apply(combine_aspect_sentiments, axis=1)

# Calculate the overall sentiment for the entire dataset
overall_sentiment = df['Overall_Sentiment'].mode().iloc[0]
print("Overall Sentiment for the Dataset:", overall_sentiment)

Overall Sentiment for the Dataset: positive
